In [67]:
#Read in individual datacubes and combine into a single dataframe
#*** Finish copying over files


from astropy.table import Table
import pandas as pd

df = pd.DataFrame()

sightline_list = ['J010619+004823','J012403+004432','J013340+040059','J013724-422417','J015741-010629',
                  'J020944+051713','J024401-013403','J033413-161205','J033900-013318','J094932+033531',
                  'J095852+120245','J102009+104002','J111008+024458','J111113-080401','J120917+113830',
                  'J123055-113909']#,'J124957-015928','J133254+005250','J142438+225600','J162116-004251',
                  #'J193957-100241','J200324-325145','J205344-354652','J221527-161133','J230301-093930',
                  #'J231543+145606','J233446-090812','J234913-371259']


for sightline in sightline_list:
    table = Table.read('/home/emma/Documents/laeML/catalogues/individual_cubes/{}_cubex_data.fits'.format(sightline))
    table['sightline'] = sightline
    pandas_df = table.to_pandas()
    df = pd.concat([df,pandas_df])



In [85]:
#df.info()
#df.head()
#df.describe()
df.columns

Index([u'id', u'n_spax_thresh', u'x_geow', u'y_geow', u'z_geow', u'x_fluxw',
       u'y_fluxw', u'z_fluxw', u'x_min', u'y_min', u'z_min', u'x_max',
       u'y_max', u'z_max', u'area_isoproj', u'area_z', u'IsoFlux', u'IsoErr',
       u'flux_minmax', u'err_minmax', u'flux_aper', u'err_aper', u'ra_geow',
       u'Dec_geow', u'lambda_geow', u'ra_fluxw', u'Dec_fluxw', u'lambda_fluxw',
       u'assoc_Id', u'SNR', u'SNR_odd', u'SNR_even', u'SNR_med', u'covfac',
       u'confidence', u'veloffset', u'EODeltaSN', u'BoxFraction',
       u'OverContinuum', u'redshift', u'CogFlux', u'CogErr', u'CogRad',
       u'type', u'notes', u'inspect', u'sightline', u'isLAE'],
      dtype='object')

In [77]:
# Check number of LAEs
nLAE = df['type'].str.contains('LAE').sum()
print(nLAE)


In [ ]:
# Create new column to show if entry is an LAE
df['isLAE'] = np.where(df['type'].str.contains('LAE'), 1, 0)

In [83]:
LAEs = df[df['isLAE']==1]

In [87]:
from sklearn.model_selection import train_test_split

# Select features
cols_to_use = ['n_spax_thresh','area_isoproj', 'area_z','IsoFlux','IsoErr','flux_minmax','lambda_geow',
               'lambda_fluxw','SNR','veloffset','EODeltaSN','BoxFraction','OverContinuum','redshift',
               'CogFlux','CogErr','CogRad']

X = df[cols_to_use]

# Select target
y = df.isLAE

# Separate data into training and validation sets
#X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [91]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
#from sklearn.impute import SimpleImputer
from sklearn.preprocessing.imputation import Imputer

my_pipeline = Pipeline(steps=[('preprocessor', Imputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [92]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

('MAE scores:\n', array([0.00058087, 0.00057643, 0.00012194, 0.00045455, 0.00033703]))


In [95]:
model = my_pipeline.fit(X,y)

In [96]:
preds = my_pipeline.predict(X)

In [97]:
preds


array([0., 0., 0., ..., 0., 0., 0.])

In [99]:
len(preds[preds==1])

638